<a href="https://colab.research.google.com/github/priyal6/NLP-Prac/blob/main/prefill_%26_decode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
prompt = "Explain retrieval-augmented generation in one sentence.\n\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Forward pass through the full prompt
# This builds the KV cache
with torch.no_grad():
    prefill_outputs = model(
        **inputs,
        use_cache=True
    )

kv_cache = prefill_outputs.past_key_values

# Start decoding with the last token of the prompt
last_token = inputs["input_ids"][:, -1:]

#decoding
generated_tokens = []
max_new_tokens = 30

with torch.no_grad():
    for _ in range(max_new_tokens):
        outputs = model(
            input_ids=last_token,
            past_key_values=kv_cache,
            use_cache=True
        )

        # Extract updated KV cache for next step
        kv_cache = outputs.past_key_values

        # Greedy decoding (choose highest prob token)
        next_token = torch.argmax(outputs.logits[:, -1, :], dim=-1, keepdim=True)

        generated_tokens.append(next_token.item())

        # Update last_token for next iteration
        last_token = next_token

        # Stop at EOS if needed
        if next_token.item() == tokenizer.eos_token_id:
            break

# Decode output
decoded_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("Generated:", decoded_text)

Generated: Retrieval-augmented generation (RAG) is a technique that uses a pre-trained language model to generate new sentences based on
